In [1]:
# from eos import proj_root
from eos.data_io.struct.eos_data import StateSpecs

2023-09-28 13:41:56.819539: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 13:41:59.189305: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 13:41:59.209191: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-28 13:41:59.209373: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there m

In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from fastavro import writer, reader, parse_schema

In [3]:
import os
from eos import proj_root

os.chdir(proj_root / "data")
os.getcwd()

'/home/n/devel/anting_remote/eos/data'

In [4]:
from dataclasses import dataclass, field
from functools import reduce

from typing import Optional
import pandas as pd
import numpy as np
from ordered_set import OrderedSet
from eos.data_io.config import drivers, trucks_by_id, drivers_by_id
from datetime import datetime

a = np.array([[1, 2, 3], [4, 5, 6]])
ts = pd.Timestamp.now(tz="Asia/Shanghai")


ts_ind = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms")
ss = np.arange(12)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
df_ss = pd.DataFrame(
    {"timestep": ts_ind, "velocity": a1, "thrust": a2, "brake": a3}
)  # .set_index('timestep')
df_ss.columns.name = "qtuple"

state = df_ss.stack().swaplevel(0, 1)
state.name = "state"
state.index.names = ["rows", "idx"]
state.sort_index(inplace=True)

a = len(ss) + np.arange(15)
speed_ser = pd.Series(np.linspace(40, 60, 3), name="speed")
row_array = a.reshape(3, 5).transpose()
rows_df = pd.DataFrame(row_array)
rows_df.columns = [f"r{i}" for i in np.arange(3)]

ts_ind = ts + pd.to_timedelta(np.arange(5 * 20, 8 * 20, 20), "ms")
ts_ser = pd.Series(ts_ind, name="timestep")
throttle_ser = pd.Series(np.linspace(0, 1.0, 5), name="throttle")
# throttle_ser
dfs = [rows_df, ts_ser, speed_ser, throttle_ser]
action = (
    reduce(
        lambda left, right: pd.merge(
            left, right, how="outer", left_index=True, right_index=True
        ),
        dfs,
    )
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)

action.name = "action"
action.index.names = ["rows", "idx"]

reward = (
    pd.DataFrame({"work": len(ss) + len(a), "timestep": ts_ind[0]}, index=[0])
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
reward.index.names = ["rows", "idx"]
reward.name = "reward"
ts_ind = ts + pd.to_timedelta(5, "s") + pd.to_timedelta(np.arange(0, 4 * 20, 20), "ms")
ss = (
    np.arange(12) + len(ss) + len(a) + len(reward) - 1
)  # exclude the timestamp in reward
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]

nstate = (
    pd.DataFrame({"timestep": ts_ind, "velocity": a1, "thrust": a2, "brake": a3})
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
nstate.name = "nstate"
nstate.index.names = ["rows", "idx"]

timestamp = pd.Series([ts], name="timestamp")
timestamp.index = pd.MultiIndex.from_product(
    [timestamp.index, [0]], names=["rows", "idx"]
)

timestamp_index = (timestamp.name, "", 0)
state_index = [(state.name, *i) for i in state.index]
reward_index = [(reward.name, *i) for i in reward.index]
action_index = [(action.name, *i) for i in action.index]
nstate_index = [(nstate.name, *i) for i in nstate.index]
multiindex = pd.MultiIndex.from_tuples(
    [timestamp_index, *state_index, *action_index, *reward_index, *nstate_index]
)
observation_list = [timestamp, state, action, reward, nstate]
observation = pd.concat(observation_list)
observation.index = multiindex

observation0 = observation.copy()
observation0.loc["timestamp", "", 0] = ts + pd.Timedelta(1, "h")
observation1 = observation.copy()
observation1.loc["timestamp", "", 0] = ts + pd.Timedelta(2, "h")
observation2 = observation.copy()
observation2.loc["timestamp", "", 0] = ts + pd.Timedelta(3, "h")
observation3 = observation.copy()
observation3.loc["timestamp", "", 0] = ts + pd.Timedelta(4, "h")
observation4 = observation.copy()
observation4.loc["timestamp", "", 0] = ts + pd.Timedelta(5, "h")
observation_list = [
    observation0,
    observation1,
    observation2,
    observation3,
    observation4,
]

dfs_epi = pd.concat(observation_list, axis=1).transpose()
dfs_epi.columns.names = ["qtuple", "rows", "idx"]

dfs_episode = dfs_epi.copy()
dfs_episode.set_index(("timestamp", "", 0), inplace=True)
dfs_episode.sort_index(axis=1, inplace=True)
dfs_episode.index.name = "timestamp"

state_cols_float = [("state", col) for col in ["brake", "thrust", "velocity"]]
action_cols_float = [("action", col) for col in ["r0", "r1", "r2", "speed", "throttle"]]
reward_cols_float = [("reward", "work")]
nstate_cols_float = [("nstate", col) for col in ["brake", "thrust", "velocity"]]
for col in action_cols_float + state_cols_float + reward_cols_float + nstate_cols_float:
    dfs_episode[col[0], col[1]] = dfs_episode[col[0], col[1]].astype(
        "float"
    )  # float16 not allowed in parquet
dfs_episode = pd.concat(
    [dfs_episode], keys=[drivers_by_id["wang-cheng"].pid], names=["driver"]
)
dfs_episode = pd.concat(
    [dfs_episode], keys=[trucks_by_id["VB7"].vid], names=["vehicle"]
)
dfs_episode = pd.concat([dfs_episode], keys=[ts], names=["episodestart"])
dfs_episode = dfs_episode.swaplevel(1, 0, axis=0)
dfs_episode = dfs_episode.swaplevel(1, 2, axis=0)
dfs_episode.sort_index(inplace=True)
dfs_episode

qtuple                                                                               action  \
rows                                                                                     r0   
idx                                                                                       0   
vehicle driver     episodestart                     timestamp                                 
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00   12.0   
                                                    2023-09-28 15:41:59.237642+08:00   12.0   
                                                    2023-09-28 16:41:59.237642+08:00   12.0   
                                                    2023-09-28 17:41:59.237642+08:00   12.0   
                                                    2023-09-28 18:41:59.237642+08:00   12.0   

qtuple                                                                                      \
rows                                                                                         
idx                                                                                      1   
vehicle driver     episodestart                     timestamp                                
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  13.0   
                                                    2023-09-28 15:41:59.237642+08:00  13.0   
                                                    2023-09-28 16:41:59.237642+08:00  13.0   
                                                    2023-09-28 17:41:59.237642+08:00  13.0   
                                                    2023-09-28 18:41:59.237642+08:00  13.0   

qtuple                                                                                      \
rows                                                                                         
idx                                                                                      2   
vehicle driver     episodestart                     timestamp                                
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  14.0   
                                                    2023-09-28 15:41:59.237642+08:00  14.0   
                                                    2023-09-28 16:41:59.237642+08:00  14.0   
                                                    2023-09-28 17:41:59.237642+08:00  14.0   
                                                    2023-09-28 18:41:59.237642+08:00  14.0   

qtuple                                                                                      \
rows                                                                                         
idx                                                                                      3   
vehicle driver     episodestart                     timestamp                                
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  15.0   
                                                    2023-09-28 15:41:59.237642+08:00  15.0   
                                                    2023-09-28 16:41:59.237642+08:00  15.0   
                                                    2023-09-28 17:41:59.237642+08:00  15.0   
                                                    2023-09-28 18:41:59.237642+08:00  15.0   

qtuple                                                                                      \
rows                                                                                         
idx                                                                                      4   
vehicle driver     episodestart                     timestamp                                
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  16.0   
                                                    2023-09-28 15:41:59.237642+08:00  16.0   
                                       

In [5]:
ts_new = pd.to_datetime((datetime.now()))
episodestart = ts_new - pd.Timedelta(2, "d")

dfs_episode0 = dfs_episode.copy()
dfs_episode0.index = dfs_episode0.index.set_levels([episodestart], level="episodestart")
dfs_episode0.index = dfs_episode0.index.set_levels(
    [[trucks_by_id["VB7"].vid], [drivers_by_id["zheng-longfei"].pid]],
    level=["vehicle", "driver"],
    verify_integrity=False,
)
ts_index = dfs_episode0.index.unique(level="timestamp")
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num - 1)
ts_index_to_drop = np.random.choice(ts_index, drop_num, replace=False)
dfs_episode0 = dfs_episode0.drop(index=ts_index_to_drop, level="timestamp")
# srs_episode0 = dfs_episode0.stack(level=['qtuple', 'rows', 'idx'])

dfs_episode1 = dfs_episode.copy()
dfs_episode1.index = dfs_episode1.index.set_levels(
    [episodestart - pd.Timedelta(3, "d")], level="episodestart"
)
dfs_episode1.index = dfs_episode1.index.set_levels(
    [[trucks_by_id["MP73"].vid], [drivers_by_id["wang-cheng"].pid]],
    level=["vehicle", "driver"],
    verify_integrity=False,
)
ts_index = dfs_episode1.index.unique(level="timestamp")
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num - 1)
ts_index_to_drop = np.random.choice(ts_index, drop_num, replace=False)
dfs_episode1 = dfs_episode1.drop(index=ts_index_to_drop, level="timestamp")
# srs_episode1 = dfs_episode1.stack(level=['qtuple', 'rows', 'idx'])

dfs_episode2 = dfs_episode.copy()
dfs_episode2.index = dfs_episode2.index.set_levels(
    [episodestart - pd.Timedelta(4, "d")], level="episodestart"
)
dfs_episode2.index = dfs_episode2.index.set_levels(
    [[trucks_by_id["VB7"].vid], [drivers_by_id["wang-cheng"].pid]],
    level=["vehicle", "driver"],
    verify_integrity=False,
)
ts_index = dfs_episode2.index.unique(level="timestamp")
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num - 1)
ts_index_to_drop = np.random.choice(ts_index, drop_num, replace=False)
dfs_episode2 = dfs_episode2.drop(index=ts_index_to_drop, level="timestamp")
# srs_episode2 = dfs_episode2.stack(level=['qtuple', 'rows', 'idx'])

dfs_episode3 = dfs_episode.copy()
dfs_episode3.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(5, "d")], level="episodestart"
)
dfs_episode3.index = dfs_episode3.index.set_levels(
    [[trucks_by_id["MP73"].vid], [drivers_by_id["zheng-longfei"].pid]],
    level=["vehicle", "driver"],
    verify_integrity=False,
)
ts_index = dfs_episode3.index.unique(level="timestamp")
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num - 1)
ts_index_to_drop = np.random.choice(ts_index, drop_num, replace=False)
dfs_episode3 = dfs_episode3.drop(index=ts_index_to_drop, level="timestamp")
# srs_episode3 = dfs_episode3.stack(level=['qtuple', 'rows', 'idx'])
from functools import reduce

episodes = [dfs_episode, dfs_episode0, dfs_episode1, dfs_episode2, dfs_episode3]
try:
    dfs_episode_all = reduce(
        lambda left, right,: pd.concat([left, right], axis=0), episodes
    )
    dfs_episode_all.sort_index(inplace=True)
    dfs_episode_all = dfs_episode_all[["state", "action", "reward", "nstate"]]
    dfs_episode_all

except Exception as e:
    print(e)
# dfs_episode_all.sort_index(inplace=True)
# dfs_episode_all = dfs_episode_all.swaplevel(1, 0, axis=0)
# dfs_episode_all = dfs_episode_all.swaplevel(1, 2, axis=0)
# dfs_episode_all.index
# dfs_episode_all.columns#
#

/tmp/ipykernel_800440/2012877544.py:70: RuntimeWarning: Cannot compare tz-naive and tz-aware timestamps, sort order is undefined for incomparable objects.
  lambda left, right,: pd.concat([left, right], axis=0), episodes
/tmp/ipykernel_800440/2012877544.py:70: RuntimeWarning: Cannot compare tz-naive and tz-aware timestamps, sort order is undefined for incomparable objects.
  lambda left, right,: pd.concat([left, right], axis=0), episodes
/tmp/ipykernel_800440/2012877544.py:70: RuntimeWarning: Cannot compare tz-naive and tz-aware timestamps, sort order is undefined for incomparable objects.
  lambda left, right,: pd.concat([left, right], axis=0), episodes


qtuple                                                                                  state  \
rows                                                                                    brake   
idx                                                                                         0   
vehicle driver        episodestart                     timestamp                                
MP73    wang-cheng    2023-09-23 13:41:59.297357       2023-09-28 15:41:59.237642+08:00   8.0   
                                                       2023-09-28 16:41:59.237642+08:00   8.0   
                                                       2023-09-28 17:41:59.237642+08:00   8.0   
                                                       2023-09-28 18:41:59.237642+08:00   8.0   
        zheng-longfei 2023-09-21 13:41:59.297357       2023-09-28 15:41:59.237642+08:00   8.0   
                                                       2023-09-28 16:41:59.237642+08:00   8.0   
                                                       2023-09-28 17:41:59.237642+08:00   8.0   
                                                       2023-09-28 18:41:59.237642+08:00   8.0   
VB7     wang-cheng    2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00   8.0   
                                                       2023-09-28 15:41:59.237642+08:00   8.0   
                                                       2023-09-28 16:41:59.237642+08:00   8.0   
                                                       2023-09-28 17:41:59.237642+08:00   8.0   
                                                       2023-09-28 18:41:59.237642+08:00   8.0   
                      2023-09-22 13:41:59.297357       2023-09-28 15:41:59.237642+08:00   8.0   
                                                       2023-09-28 17:41:59.237642+08:00   8.0   
        zheng-longfei 2023-09-26 13:41:59.297357       2023-09-28 15:41:59.237642+08:00   8.0   
                                                       2023-09-28 17:41:59.237642+08:00   8.0   

qtuple                                                                                        \
rows                                                                                           
idx                                                                                        1   
vehicle driver        episodestart                     timestamp                               
MP73    wang-cheng    2023-09-23 13:41:59.297357       2023-09-28 15:41:59.237642+08:00  9.0   
                                                       2023-09-28 16:41:59.237642+08:00  9.0   
                                                       2023-09-28 17:41:59.237642+08:00  9.0   
                                                       2023-09-28 18:41:59.237642+08:00  9.0   
        zheng-longfei 2023-09-21 13:41:59.297357       2023-09-28 15:41:59.237642+08:00  9.0   
                                                       2023-09-28 16:41:59.237642+08:00  9.0   
                                                       2023-09-28 17:41:59.237642+08:00  9.0   
                                                       2023-09-28 18:41:59.237642+08:00  9.0   
VB7     wang-cheng    2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  9.0   
                                                       2023-09-28 15:41:59.237642+08:00  9.0   
                                                       2023-09-28 16:41:59.237642+08:00  9.0   
                                                       2023-09-28 17:41:59.237642+08:00  9.0   
                                                       2023-09-28 18:41:59.237642+08:00  9.0   
                      2023-09-22 13:41:59.297357       2023-09-28 15:41:59.237642+08:00  9.0   
                                                       2023-09-28 17:41:59.237642+08:00  9.0   
        zheng-longfei 2023-09-26 13:41:59.297357       2023-09-28 15:41:59.237642+08:00  9.0   
                                          

In [6]:
dfs_episode
# episodes[0]

qtuple                                                                               action  \
rows                                                                                     r0   
idx                                                                                       0   
vehicle driver     episodestart                     timestamp                                 
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00   12.0   
                                                    2023-09-28 15:41:59.237642+08:00   12.0   
                                                    2023-09-28 16:41:59.237642+08:00   12.0   
                                                    2023-09-28 17:41:59.237642+08:00   12.0   
                                                    2023-09-28 18:41:59.237642+08:00   12.0   

qtuple                                                                                      \
rows                                                                                         
idx                                                                                      1   
vehicle driver     episodestart                     timestamp                                
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  13.0   
                                                    2023-09-28 15:41:59.237642+08:00  13.0   
                                                    2023-09-28 16:41:59.237642+08:00  13.0   
                                                    2023-09-28 17:41:59.237642+08:00  13.0   
                                                    2023-09-28 18:41:59.237642+08:00  13.0   

qtuple                                                                                      \
rows                                                                                         
idx                                                                                      2   
vehicle driver     episodestart                     timestamp                                
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  14.0   
                                                    2023-09-28 15:41:59.237642+08:00  14.0   
                                                    2023-09-28 16:41:59.237642+08:00  14.0   
                                                    2023-09-28 17:41:59.237642+08:00  14.0   
                                                    2023-09-28 18:41:59.237642+08:00  14.0   

qtuple                                                                                      \
rows                                                                                         
idx                                                                                      3   
vehicle driver     episodestart                     timestamp                                
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  15.0   
                                                    2023-09-28 15:41:59.237642+08:00  15.0   
                                                    2023-09-28 16:41:59.237642+08:00  15.0   
                                                    2023-09-28 17:41:59.237642+08:00  15.0   
                                                    2023-09-28 18:41:59.237642+08:00  15.0   

qtuple                                                                                      \
rows                                                                                         
idx                                                                                      4   
vehicle driver     episodestart                     timestamp                                
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  16.0   
                                                    2023-09-28 15:41:59.237642+08:00  16.0   
                                       

In [7]:
from dataclasses import asdict

indices = dfs_episode.index
# ep_start = indices.get_level_values(level='episodestart')[0]
indices_dict = [
    {indices.names[i]: level for i, level in enumerate(levels)} for levels in indices
]
episode_meta = indices_dict[0].copy()
episode_meta["episodestart"] = episode_meta["episodestart"].timestamp() * 1e6
episode_meta["timestamp"] = episode_meta["timestamp"].timestamp() * 1e6
try:
    episode_meta.pop("timestamp")
except KeyError:
    print(f"Key 'timestamp' not found")
# episode_meta

# episodes_indices
episodes_indices_dict = [
    [{indices.names[i]: level for i, level in enumerate(levels)} for levels in df.index]
    for df in episodes
]
for indices in episodes_indices_dict:
    indices[0].pop("timestamp")

episodes_meta_dict = [indices[0] for indices in episodes_indices_dict]

for ep_mt in episodes_meta_dict:
    ep_mt["episodestart"] = ep_mt["episodestart"].timestamp() * 1e6
episodes_meta_dict

1695883319237642.0

Timestamp('2023-09-28 14:41:59.237642+0800', tz='Asia/Shanghai')

Timestamp('2023-09-28 15:41:59.237642+0800', tz='Asia/Shanghai')

Timestamp('2023-09-28 15:41:59.237642+0800', tz='Asia/Shanghai')

Timestamp('2023-09-28 15:41:59.237642+0800', tz='Asia/Shanghai')

Timestamp('2023-09-28 15:41:59.237642+0800', tz='Asia/Shanghai')

[{'vehicle': 'VB7',
  'driver': 'wang-cheng',
  'episodestart': 1695879719237642.0},
 {'vehicle': 'VB7',
  'driver': 'zheng-longfei',
  'episodestart': 1695735719297357.0},
 {'vehicle': 'MP73',
  'driver': 'wang-cheng',
  'episodestart': 1695476519297357.0},
 {'vehicle': 'VB7',
  'driver': 'wang-cheng',
  'episodestart': 1695390119297357.0},
 {'vehicle': 'MP73',
  'driver': 'zheng-longfei',
  'episodestart': 1695303719297357.0}]

In [8]:
from eos.data_io.struct import (
    ObservationMeta,
    StateSpecs,
    StateUnitCodes,
    ActionSpecs,
    RewardSpecs,
)
from eos.data_io.utils.eos_pandas import ep_nest, df_to_ep_nested_dict, avro_ep_encoding
from eos.data_io.config import trucks_by_id

# from pydantic import schema_json_of, schema_of

observation_meta = ObservationMeta(
    state_specs=StateSpecs(
        state_unit_codes=StateUnitCodes(),
        state_number=3,
        unit_number_per_state=trucks_by_id["VB7"].tbox_unit_number,  # 4
        unit_duration=trucks_by_id["VB7"].tbox_unit_duration,  # 1s
        frequency=trucks_by_id["VB7"].tbox_signal_frequency,  # 50 hz
    ),
    action_specs=ActionSpecs(
        action_unit_code="nm",
        action_row_number=trucks_by_id["VB7"].torque_table_row_num_flash,
        action_column_number=len(trucks_by_id["VB7"].pedal_scale),
    ),
    reward_specs=RewardSpecs(reward_unit_code="wh", reward_number=1),
    site=trucks_by_id["VB7"].site,
)


dict_nested = avro_ep_encoding(dfs_episode)
dict_nested

[{'timestamp': 1695883319237642.0,
  'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
   'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
   'r2': [22.0, 23.0, 24.0, 25.0, 26.0],
   'speed': [40.0, 50.0, 60.0],
   'throttle': [0.0, 0.25, 0.5, 0.75, 1.0],
   'timestep': [1695879719337642.0, 1695879719357642.0, 1695879719377642.0]},
  'nstate': {'brake': [36.0, 37.0, 38.0, 39.0],
   'thrust': [32.0, 33.0, 34.0, 35.0],
   'timestep': [1695879724237642.0,
    1695879724257642.0,
    1695879724277642.0,
    1695879724297642.0],
   'velocity': [28.0, 29.0, 30.0, 31.0]},
  'reward': {'timestep': [1695879719337642.0], 'work': [27.0]},
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1695879719237642.0,
    1695879719257642.0,
    1695879719277642.0,
    1695879719297642.0],
   'velocity': [0.0, 1.0, 2.0, 3.0]}},
 {'timestamp': 1695886919237642.0,
  'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
   'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
   'r2': [22.0, 23.0,

In [9]:
dict_ep = {
    "episodestart": episode_meta["episodestart"],
    "meta": {
        "episode_meta": episode_meta,
        "observation_meta": observation_meta.model_dump(),
    },
    "sequence": dict_nested,
}
dict_ep

{'episodestart': 1695879719237642.0,
 'meta': {'episode_meta': {'vehicle': 'VB7',
   'driver': 'wang-cheng',
   'episodestart': 1695879719237642.0},
  'observation_meta': {'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
     'thrust_unit_code': 'pct',
     'brake_unit_code': 'pct'},
    'state_number': 3,
    'unit_number_per_state': 4,
    'unit_duration': 1.0,
    'frequency': 50},
   'action_specs': {'action_unit_code': 'nm',
    'action_row_number': 4,
    'action_column_number': 17},
   'reward_specs': {'reward_unit_code': 'wh', 'reward_number': 1},
   'site': 'anting'}},
 'sequence': [{'timestamp': 1695883319237642.0,
   'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
    'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
    'r2': [22.0, 23.0, 24.0, 25.0, 26.0],
    'speed': [40.0, 50.0, 60.0],
    'throttle': [0.0, 0.25, 0.5, 0.75, 1.0],
    'timestep': [1695879719337642.0, 1695879719357642.0, 1695879719377642.0]},
   'nstate': {'brake': [36.0, 37.0, 38.0, 39.0],
    'thr

In [10]:
episodes_dict_nested = [avro_ep_encoding(ep) for ep in episodes]
episodes_dict_nested[0]

[{'timestamp': 1695883319237642.0,
  'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
   'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
   'r2': [22.0, 23.0, 24.0, 25.0, 26.0],
   'speed': [40.0, 50.0, 60.0],
   'throttle': [0.0, 0.25, 0.5, 0.75, 1.0],
   'timestep': [1695879719337642.0, 1695879719357642.0, 1695879719377642.0]},
  'nstate': {'brake': [36.0, 37.0, 38.0, 39.0],
   'thrust': [32.0, 33.0, 34.0, 35.0],
   'timestep': [1695879724237642.0,
    1695879724257642.0,
    1695879724277642.0,
    1695879724297642.0],
   'velocity': [28.0, 29.0, 30.0, 31.0]},
  'reward': {'timestep': [1695879719337642.0], 'work': [27.0]},
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1695879719237642.0,
    1695879719257642.0,
    1695879719277642.0,
    1695879719297642.0],
   'velocity': [0.0, 1.0, 2.0, 3.0]}},
 {'timestamp': 1695886919237642.0,
  'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
   'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
   'r2': [22.0, 23.0,

In [11]:
dict_nested_states = [[step["state"] for step in ep] for ep in episodes_dict_nested]
dict_nested_nstates = [[step["nstate"] for step in ep] for ep in episodes_dict_nested]
dict_nested_rewards = [[step["reward"] for step in ep] for ep in episodes_dict_nested]
dict_nested_actions = [[step["action"] for step in ep] for ep in episodes_dict_nested]
dict_nested_timestamps = [
    [step["timestamp"] for step in ep] for ep in episodes_dict_nested
]

arr_states = [
    [{"ts": ts, "state": state_arr[i]} for i, ts in enumerate(ts_arr)]
    for (ts_arr, state_arr) in zip(dict_nested_timestamps, dict_nested_states)
]
arr_states[0]
# dict_nested_timestamps[0]

[{'ts': 1695883319237642.0,
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1695879719237642.0,
    1695879719257642.0,
    1695879719277642.0,
    1695879719297642.0],
   'velocity': [0.0, 1.0, 2.0, 3.0]}},
 {'ts': 1695886919237642.0,
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1695879719237642.0,
    1695879719257642.0,
    1695879719277642.0,
    1695879719297642.0],
   'velocity': [0.0, 1.0, 2.0, 3.0]}},
 {'ts': 1695890519237642.0,
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1695879719237642.0,
    1695879719257642.0,
    1695879719277642.0,
    1695879719297642.0],
   'velocity': [0.0, 1.0, 2.0, 3.0]}},
 {'ts': 1695894119237642.0,
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1695879719237642.0,
    1695879719257642.0,
    1695879719277642.0,
    1695879719297642.0],
   'velocity': 

In [12]:
import fastavro
import json

state_unit_fields_schema = [
    {"name": "velocity_unit_code", "type": "string"},
    {"name": "thrust_unit_code", "type": "string"},
    {"name": "brake_unit_code", "type": "string"},
]
state_specs_fields_schema = [
    {
        "name": "state_unit_codes",
        "type": {
            "type": "record",
            "name": "state_unit_codes_",
            "fields": state_unit_fields_schema,
        },
    },
    {"name": "state_number", "type": "int"},
    {"name": "unit_number_per_state", "type": "int"},
    {"name": "unit_duration", "type": "int"},
    {"name": "frequency", "type": "int"},
]
action_specs_fields_schema = [
    {"name": "action_unit_code", "type": "string"},
    {"name": "action_row_number", "type": "int"},
    {"name": "action_column_number", "type": "int"},
]
reward_specs_fields_schema = [{"name": "reward_unit_code", "type": "string"}]

episode_meta_fields_schema = [
    {"name": "vehicle", "type": "string"},
    {"name": "driver", "type": "string"},
    {"name": "episodestart", "type": "long", "logicalType": "timestamp-micros"},
]
observation_meta_fields_schema = [
    {
        "name": "state_specs",
        "type": {
            "type": "record",
            "name": "state_specs_",
            "fields": state_specs_fields_schema,
        },
    },
    {
        "name": "action_specs",
        "type": {
            "type": "record",
            "name": "action_specs_",
            "fields": action_specs_fields_schema,
        },
    },
    {
        "name": "reward_specs",
        "type": {
            "type": "record",
            "name": "reward_specs_",
            "fields": reward_specs_fields_schema,
        },
    },
    {"name": "site", "type": "string"},
]

state_fields_schema = [
    {"name": "velocity", "type": {"type": "array", "items": "float"}},
    {"name": "thrust", "type": {"type": "array", "items": "float"}},
    {"name": "brake", "type": {"type": "array", "items": "float"}},
    {
        "name": "timestep",
        "type": {
            "type": "array",
            "items": {"type": "long", "logicalType": "timestamp-micros"},
        },
    },
]

torque_table_row_names = ["r0", "r1", "r2"]
action_fields_schema = [
    {"name": r, "type": {"type": "array", "items": "float"}}
    for r in torque_table_row_names
]
action_fields_schema += [
    {"name": "speed", "type": {"type": "array", "items": "float"}},
    {"name": "throttle", "type": {"type": "array", "items": "float"}},
    {
        "name": "timestep",
        "type": {
            "type": "array",
            "items": {"type": "long", "logicalType": "timestamp-micros"},
        },
    },
]


reward_fields_schema = [
    {"name": "work", "type": {"type": "array", "items": "float"}},
    {
        "name": "timestep",
        "type": {
            "type": "array",
            "items": {"type": "long", "logicalType": "timestamp-micros"},
        },
    },
]


episode_array_fields_schema = [
    {
        "type": "long",
        "name": "timestamp",
        "logicalType": "timestamp-micros",
    },
    {
        "name": "state",
        "type": {
            "type": "record",
            "name": "state_",
            "fields": state_fields_schema,
        },
    },
    {
        "name": "action",
        "type": {
            "type": "record",
            "name": "action_",
            "fields": action_fields_schema,
        },
    },
    {
        "name": "reward",
        "type": {
            "type": "record",
            "name": "reward_",
            "fields": reward_fields_schema,
        },
    },
    {
        "name": "nstate",
        "type": {
            "type": "record",
            "name": "nstate_",
            "fields": state_fields_schema,
        },
    },
]

schema_episode = {
    "type": "record",
    "name": "episode",
    "doc": "episode data with a timestamp, meta description and an array of episode steps",
    "fields": [
        {"type": "long", "name": "episodestart", "logicalType": "timestamp-micros"},
        {
            "name": "meta",
            "type": {
                "type": "record",
                "name": "meta_",
                "fields": [
                    {
                        "name": "episode_meta",
                        "type": {
                            "type": "record",
                            "name": "episode_meta_",
                            "fields": episode_meta_fields_schema,
                        },
                    },
                    {
                        "name": "observation_meta",
                        "type": {
                            "type": "record",
                            "name": "observation_meta_",
                            "fields": observation_meta_fields_schema,
                        },
                    },
                ],
            },
        },
        {
            "name": "sequence",
            "type": {
                "type": "array",
                "items": {
                    "name": "step",  # not used in constructing the episode observation array data
                    "type": "record",
                    "fields": episode_array_fields_schema,
                },
            },
        },
    ],
}

parsed_schema_episode = fastavro.schema.parse_schema(schema_episode)
print(json.dumps(schema_episode, indent=2))

{
  "type": "record",
  "name": "episode",
  "doc": "episode data with a timestamp, meta description and an array of episode steps",
  "fields": [
    {
      "type": "long",
      "name": "episodestart",
      "logicalType": "timestamp-micros"
    },
    {
      "name": "meta",
      "type": {
        "type": "record",
        "name": "meta_",
        "fields": [
          {
            "name": "episode_meta",
            "type": {
              "type": "record",
              "name": "episode_meta_",
              "fields": [
                {
                  "name": "vehicle",
                  "type": "string"
                },
                {
                  "name": "driver",
                  "type": "string"
                },
                {
                  "name": "episodestart",
                  "type": "long",
                  "logicalType": "timestamp-micros"
                }
              ]
            }
          },
          {
            "name": "observati

In [13]:
# records_episodes = [
#     {u'episodestart': episodes_indices_dict[0]['episodestart'],
#      u'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
#      u'sequence': episodes_dict_nested[0]},
#     {u'episodestart': episodes_indices_dict[1]['episodestart'],
#      u'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
#      u'sequence': episodes_dict_nested[1]},
#     {u'episodestart': episodes_indices_dict[2]['episodestart'],
#      u'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
#      u'sequence': episodes_dict_nested[2]},
#     {u'episodestart': episodes_indices_dict[3]['episodestart'],
#      u'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
#      u'sequence': episodes_dict_nested[3]},
#     {u'episodestart': episodes_indices_dict[4]['episodestart'],
#      u'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
#      u'sequence': episodes_dict_nested[4]},
# ]


records_episodes = [
    {
        "episodestart": episodes_meta_dict[i]["episodestart"],
        "meta": {
            "episode_meta": episodes_meta_dict[i],
            "observation_meta": observation_meta.dict(),
        },
        "sequence": episodes_dict_nested[i],
    }
    for i in range(4)
]

records_episodes[1]

/tmp/ipykernel_800440/605212411.py:21: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.1.1/migration/
  'meta': {'episode_meta': episodes_meta_dict[i], 'observation_meta': observation_meta.dict()},


{'episodestart': 1695735719297357.0,
 'meta': {'episode_meta': {'vehicle': 'VB7',
   'driver': 'zheng-longfei',
   'episodestart': 1695735719297357.0},
  'observation_meta': {'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
     'thrust_unit_code': 'pct',
     'brake_unit_code': 'pct'},
    'state_number': 3,
    'unit_number_per_state': 4,
    'unit_duration': 1.0,
    'frequency': 50},
   'action_specs': {'action_unit_code': 'nm',
    'action_row_number': 4,
    'action_column_number': 17},
   'reward_specs': {'reward_unit_code': 'wh', 'reward_number': 1},
   'site': 'anting'}},
 'sequence': [{'timestamp': 1695886919237642.0,
   'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
    'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
    'r2': [22.0, 23.0, 24.0, 25.0, 26.0],
    'speed': [40.0, 50.0, 60.0],
    'throttle': [0.0, 0.25, 0.5, 0.75, 1.0],
    'timestep': [1695879719337642.0, 1695879719357642.0, 1695879719377642.0]},
   'nstate': {'brake': [36.0, 37.0, 38.0, 39.0],
    '

# Write the episodes to a bag

In [14]:
os.getcwd()
# os.chdir('bags')
# os.getcwd()

'/home/n/devel/anting_remote/eos/data'

In [15]:
import dask.bag as db

b_episodes = db.from_sequence(records_episodes)
print(b_episodes.npartitions)
data = b_episodes.take(4, npartitions=4)
len(data)
b_episodes.to_avro("bag_episodes1.*.avro", schema=parsed_schema_episode)

4


4

['/home/n/devel/anting_remote/eos/data/bag_episodes1.0.avro',
 '/home/n/devel/anting_remote/eos/data/bag_episodes1.1.avro',
 '/home/n/devel/anting_remote/eos/data/bag_episodes1.2.avro',
 '/home/n/devel/anting_remote/eos/data/bag_episodes1.3.avro']

In [16]:
b_episodes_read = db.read_avro("bag_episodes1.*.avro")
rec = b_episodes_read.take(4, npartitions=4)
len(rec)
print(rec[0]["meta"]["episode_meta"]["vehicle"])
print(rec[0]["meta"]["episode_meta"]["driver"])
print(rec[0]["sequence"][0])

4

VB7
wang-cheng
{'timestamp': 1695883319237642, 'state': {'velocity': [0.0, 1.0, 2.0, 3.0], 'thrust': [4.0, 5.0, 6.0, 7.0], 'brake': [8.0, 9.0, 10.0, 11.0], 'timestep': [datetime.datetime(2023, 9, 28, 5, 41, 59, 237642, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 9, 28, 5, 41, 59, 257642, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 9, 28, 5, 41, 59, 277642, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 9, 28, 5, 41, 59, 297642, tzinfo=datetime.timezone.utc)]}, 'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0], 'r1': [17.0, 18.0, 19.0, 20.0, 21.0], 'r2': [22.0, 23.0, 24.0, 25.0, 26.0], 'speed': [40.0, 50.0, 60.0], 'throttle': [0.0, 0.25, 0.5, 0.75, 1.0], 'timestep': [datetime.datetime(2023, 9, 28, 5, 41, 59, 337642, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 9, 28, 5, 41, 59, 357642, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 9, 28, 5, 41, 59, 377642, tzinfo=datetime.timezone.utc)]}, 'reward': {'work': [27.0], 'timestep': [datetime.dat

# Add an episode to an existing bag

In [17]:
records_episode_to_add = [
    {
        "episodestart": episodes_meta_dict[4]["episodestart"],
        "meta": {
            "episode_meta": episodes_meta_dict[4],
            "observation_meta": observation_meta.model_dump(),
        },
        "sequence": episodes_dict_nested[4],
    }
]
# records_episode_to_add

b_episodes_new = db.concat([b_episodes_read, db.from_sequence(records_episode_to_add)])
print(b_episodes_new.npartitions)
b_episodes_new.to_avro("bag_episodes.*.avro", schema=parsed_schema_episode)

5


['/home/n/devel/anting_remote/eos/data/bag_episodes.0.avro',
 '/home/n/devel/anting_remote/eos/data/bag_episodes.1.avro',
 '/home/n/devel/anting_remote/eos/data/bag_episodes.2.avro',
 '/home/n/devel/anting_remote/eos/data/bag_episodes.3.avro',
 '/home/n/devel/anting_remote/eos/data/bag_episodes.4.avro']

In [18]:
b_episodes_read = db.read_avro("bag_episodes.*.avro")
print(b_episodes_read.npartitions)
b_episodes_read.take(6, npartitions=6)

8


/dpt/micromamba/envs/meos/lib/python3.10/site-packages/dask/bag/core.py:2511: UserWarning: Insufficient elements for `take`. 6 elements requested, only 5 elements available. Try passing larger `npartitions` to `take`.
  warnings.warn(


({'episodestart': 1695879719237642,
  'meta': {'episode_meta': {'vehicle': 'VB7',
    'driver': 'wang-cheng',
    'episodestart': 1695879719237642},
   'observation_meta': {'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
      'thrust_unit_code': 'pct',
      'brake_unit_code': 'pct'},
     'state_number': 3,
     'unit_number_per_state': 4,
     'unit_duration': 1,
     'frequency': 50},
    'action_specs': {'action_unit_code': 'nm',
     'action_row_number': 4,
     'action_column_number': 17},
    'reward_specs': {'reward_unit_code': 'wh'},
    'site': 'anting'}},
  'sequence': [{'timestamp': 1695883319237642,
    'state': {'velocity': [0.0, 1.0, 2.0, 3.0],
     'thrust': [4.0, 5.0, 6.0, 7.0],
     'brake': [8.0, 9.0, 10.0, 11.0],
     'timestep': [datetime.datetime(2023, 9, 28, 5, 41, 59, 237642, tzinfo=datetime.timezone.utc),
      datetime.datetime(2023, 9, 28, 5, 41, 59, 257642, tzinfo=datetime.timezone.utc),
      datetime.datetime(2023, 9, 28, 5, 41, 59, 2776

In [19]:
an_episode = b_episodes_read.take(2, npartitions=6)
type(an_episode[0])
len(an_episode)
ep_meta = an_episode[0]["meta"]["episode_meta"]
obs_meta = an_episode[0]["meta"]["observation_meta"]

obs_meta1 = an_episode[1]["meta"]["observation_meta"]
for key, val in (ep_meta | obs_meta).items():
    print(key, val)

obs_meta

dict

2

vehicle VB7
driver wang-cheng
episodestart 1695879719237642
state_specs {'state_unit_codes': {'velocity_unit_code': 'kph', 'thrust_unit_code': 'pct', 'brake_unit_code': 'pct'}, 'state_number': 3, 'unit_number_per_state': 4, 'unit_duration': 1, 'frequency': 50}
action_specs {'action_unit_code': 'nm', 'action_row_number': 4, 'action_column_number': 17}
reward_specs {'reward_unit_code': 'wh'}
site anting


{'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
   'thrust_unit_code': 'pct',
   'brake_unit_code': 'pct'},
  'state_number': 3,
  'unit_number_per_state': 4,
  'unit_duration': 1,
  'frequency': 50},
 'action_specs': {'action_unit_code': 'nm',
  'action_row_number': 4,
  'action_column_number': 17},
 'reward_specs': {'reward_unit_code': 'wh'},
 'site': 'anting'}

In [20]:
{**obs_meta}

{'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
   'thrust_unit_code': 'pct',
   'brake_unit_code': 'pct'},
  'state_number': 3,
  'unit_number_per_state': 4,
  'unit_duration': 1,
  'frequency': 50},
 'action_specs': {'action_unit_code': 'nm',
  'action_row_number': 4,
  'action_column_number': 17},
 'reward_specs': {'reward_unit_code': 'wh'},
 'site': 'anting'}

In [21]:
from eos.data_io.struct.eos_data import ObservationMeta

meta1 = ObservationMeta(**obs_meta1)
meta = ObservationMeta(**obs_meta)
meta1.have_same_meta(meta)
meta1.model_dump()

True

{'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
   'thrust_unit_code': 'pct',
   'brake_unit_code': 'pct'},
  'state_number': 3,
  'unit_number_per_state': 4,
  'unit_duration': 1.0,
  'frequency': 50},
 'action_specs': {'action_unit_code': 'nm',
  'action_row_number': 4,
  'action_column_number': 17},
 'reward_specs': {'reward_unit_code': 'wh', 'reward_number': 1},
 'site': 'anting'}

In [22]:
b_episodes_read = db.read_avro("bag_episodes.*.avro")
print(b_episodes_read.npartitions)
b_episodes_read.take(6, npartitions=6)

8


/dpt/micromamba/envs/meos/lib/python3.10/site-packages/dask/bag/core.py:2511: UserWarning: Insufficient elements for `take`. 6 elements requested, only 5 elements available. Try passing larger `npartitions` to `take`.
  warnings.warn(


({'episodestart': 1695879719237642,
  'meta': {'episode_meta': {'vehicle': 'VB7',
    'driver': 'wang-cheng',
    'episodestart': 1695879719237642},
   'observation_meta': {'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
      'thrust_unit_code': 'pct',
      'brake_unit_code': 'pct'},
     'state_number': 3,
     'unit_number_per_state': 4,
     'unit_duration': 1,
     'frequency': 50},
    'action_specs': {'action_unit_code': 'nm',
     'action_row_number': 4,
     'action_column_number': 17},
    'reward_specs': {'reward_unit_code': 'wh'},
    'site': 'anting'}},
  'sequence': [{'timestamp': 1695883319237642,
    'state': {'velocity': [0.0, 1.0, 2.0, 3.0],
     'thrust': [4.0, 5.0, 6.0, 7.0],
     'brake': [8.0, 9.0, 10.0, 11.0],
     'timestep': [datetime.datetime(2023, 9, 28, 5, 41, 59, 237642, tzinfo=datetime.timezone.utc),
      datetime.datetime(2023, 9, 28, 5, 41, 59, 257642, tzinfo=datetime.timezone.utc),
      datetime.datetime(2023, 9, 28, 5, 41, 59, 2776

# Deduplication

In [23]:
dicts = b_episodes_read.compute()
print(f"length: {len(dicts)}")
for b in dicts:
    b["episodestart"] = pd.to_datetime(b["episodestart"], unit="us")
    print(
        f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} "
    )

length: 6
vehicle: VB7; driver: wang-cheng; episodestart: 2023-09-28 05:41:59.237642 
vehicle: VB7; driver: zheng-longfei; episodestart: 2023-09-26 13:41:59.297357 
vehicle: MP73; driver: wang-cheng; episodestart: 2023-09-23 13:41:59.297357 
vehicle: VB7; driver: wang-cheng; episodestart: 2023-09-22 13:41:59.297357 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-09-21 13:41:59.297357 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-09-19 14:54:27.750917 


In [24]:
dicts = b_episodes_read.distinct(
    lambda x: (
        x["meta"]["episode_meta"]["driver"],
        x["meta"]["episode_meta"]["vehicle"],
        x["meta"]["episode_meta"]["episodestart"],
    )
).compute()
for b in dicts:
    b["episodestart"] = pd.to_datetime(b["episodestart"], unit="us")
    print(
        f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} "
    )

vehicle: VB7; driver: wang-cheng; episodestart: 2023-09-28 05:41:59.237642 
vehicle: VB7; driver: zheng-longfei; episodestart: 2023-09-26 13:41:59.297357 
vehicle: MP73; driver: wang-cheng; episodestart: 2023-09-23 13:41:59.297357 
vehicle: VB7; driver: wang-cheng; episodestart: 2023-09-22 13:41:59.297357 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-09-21 13:41:59.297357 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-09-19 14:54:27.750917 


In [25]:
len(dicts)

6

# decode nested dicts to pandas dataframe

In [26]:
anep = dicts[0]
anep

{'episodestart': Timestamp('2023-09-28 05:41:59.237642'),
 'meta': {'episode_meta': {'vehicle': 'VB7',
   'driver': 'wang-cheng',
   'episodestart': 1695879719237642},
  'observation_meta': {'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
     'thrust_unit_code': 'pct',
     'brake_unit_code': 'pct'},
    'state_number': 3,
    'unit_number_per_state': 4,
    'unit_duration': 1,
    'frequency': 50},
   'action_specs': {'action_unit_code': 'nm',
    'action_row_number': 4,
    'action_column_number': 17},
   'reward_specs': {'reward_unit_code': 'wh'},
   'site': 'anting'}},
 'sequence': [{'timestamp': 1695883319237642,
   'state': {'velocity': [0.0, 1.0, 2.0, 3.0],
    'thrust': [4.0, 5.0, 6.0, 7.0],
    'brake': [8.0, 9.0, 10.0, 11.0],
    'timestep': [datetime.datetime(2023, 9, 28, 5, 41, 59, 237642, tzinfo=datetime.timezone.utc),
     datetime.datetime(2023, 9, 28, 5, 41, 59, 257642, tzinfo=datetime.timezone.utc),
     datetime.datetime(2023, 9, 28, 5, 41, 59, 2776

In [27]:
# anep['meta']['episode_meta']
# for k,v in anep['sequence']
len(anep["sequence"])
# anep['sequence'][0]
for qtuple, obs in anep["sequence"][0].items():
    print(f"qtuple: {qtuple};\nobs: {obs}\n\n")

5

qtuple: timestamp;
obs: 1695883319237642


qtuple: state;
obs: {'velocity': [0.0, 1.0, 2.0, 3.0], 'thrust': [4.0, 5.0, 6.0, 7.0], 'brake': [8.0, 9.0, 10.0, 11.0], 'timestep': [datetime.datetime(2023, 9, 28, 5, 41, 59, 237642, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 9, 28, 5, 41, 59, 257642, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 9, 28, 5, 41, 59, 277642, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 9, 28, 5, 41, 59, 297642, tzinfo=datetime.timezone.utc)]}


qtuple: action;
obs: {'r0': [12.0, 13.0, 14.0, 15.0, 16.0], 'r1': [17.0, 18.0, 19.0, 20.0, 21.0], 'r2': [22.0, 23.0, 24.0, 25.0, 26.0], 'speed': [40.0, 50.0, 60.0], 'throttle': [0.0, 0.25, 0.5, 0.75, 1.0], 'timestep': [datetime.datetime(2023, 9, 28, 5, 41, 59, 337642, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 9, 28, 5, 41, 59, 357642, tzinfo=datetime.timezone.utc), datetime.datetime(2023, 9, 28, 5, 41, 59, 377642, tzinfo=datetime.timezone.utc)]}


qtuple: reward;
obs: {'work': [2

In [28]:
# dict_observations = {(meta['vehicle'], meta['driver'], meta['episodestart'],
#                       pd.to_datetime(obs['timestamp'],unit='us'), qtuple): value
#                         for meta, obs in zip(anep['meta']['episode_meta'],anep['sequence'])
#                         for qtuple, value in obs.items() if qtuple != 'timestamp'}

dict_observations = [
    {
        (
            anep["meta"]["episode_meta"]["vehicle"],
            anep["meta"]["episode_meta"]["driver"],
            pd.to_datetime(
                anep["meta"]["episode_meta"]["episodestart"], unit="us", utc=True
            ).tz_convert("Asia/Shanghai"),
            pd.to_datetime(step["timestamp"], unit="us", utc=True).tz_convert(
                "Asia/Shanghai"
            ),
            qtuple,
            rows,
            idx,
        ): item
        if rows != "timestep"
        else pd.to_datetime(item).tz_convert("Asia/Shanghai")
        for qtuple, obs in step.items()
        if qtuple != "timestamp"
        for rows, value in obs.items()
        for idx, item in enumerate(value)
    }
    for step in anep["sequence"]
]

dict_ep = {k: v for d in dict_observations for k, v in d.items()}


# d = dict_observations[0]
ser_decoded = pd.Series(dict_ep)
ser_decoded.index.names = [
    "vehicle",
    "driver",
    "episodestart",
    "timestamp",
    "qtuple",
    "rows",
    "idx",
]
df_decoded = ser_decoded.unstack(level=["qtuple", "rows", "idx"])
df_decoded

qtuple                                                                                  state  \
rows                                                                                 velocity   
idx                                                                                         0   
vehicle driver     episodestart                     timestamp                                   
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00      0.0   
                                                    2023-09-28 15:41:59.237642+08:00      0.0   
                                                    2023-09-28 16:41:59.237642+08:00      0.0   
                                                    2023-09-28 17:41:59.237642+08:00      0.0   
                                                    2023-09-28 18:41:59.237642+08:00      0.0   

qtuple                                                                                     \
rows                                                                                        
idx                                                                                     1   
vehicle driver     episodestart                     timestamp                               
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  1.0   
                                                    2023-09-28 15:41:59.237642+08:00  1.0   
                                                    2023-09-28 16:41:59.237642+08:00  1.0   
                                                    2023-09-28 17:41:59.237642+08:00  1.0   
                                                    2023-09-28 18:41:59.237642+08:00  1.0   

qtuple                                                                                     \
rows                                                                                        
idx                                                                                     2   
vehicle driver     episodestart                     timestamp                               
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  2.0   
                                                    2023-09-28 15:41:59.237642+08:00  2.0   
                                                    2023-09-28 16:41:59.237642+08:00  2.0   
                                                    2023-09-28 17:41:59.237642+08:00  2.0   
                                                    2023-09-28 18:41:59.237642+08:00  2.0   

qtuple                                                                                     \
rows                                                                                        
idx                                                                                     3   
vehicle driver     episodestart                     timestamp                               
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  3.0   
                                                    2023-09-28 15:41:59.237642+08:00  3.0   
                                                    2023-09-28 16:41:59.237642+08:00  3.0   
                                                    2023-09-28 17:41:59.237642+08:00  3.0   
                                                    2023-09-28 18:41:59.237642+08:00  3.0   

qtuple                                                                                       \
rows                                                                                 thrust   
idx                                                                                       0   
vehicle driver     episodestart                     timestamp                                 
VB7     wang-cheng 2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00    4.0   
                                                    2023-09-28 15:41:59.237642+08:00    4.0   
                                          

In [29]:
len(dicts)

6

In [30]:
from functools import reduce

df_episodes_list = []
for ep in dicts:
    dict_observations = [
        {
            (
                ep["meta"]["episode_meta"]["vehicle"],
                ep["meta"]["episode_meta"]["driver"],
                pd.to_datetime(
                    ep["meta"]["episode_meta"]["episodestart"], unit="us", utc=True
                ).tz_convert("Asia/Shanghai"),
                pd.to_datetime(step["timestamp"], unit="us", utc=True).tz_convert(
                    "Asia/Shanghai"
                ),
                qtuple,
                rows,
                idx,
            ): item
            if rows != "timestep"
            else pd.to_datetime(item).tz_convert("Asia/Shanghai")
            for qtuple, obs in step.items()
            if qtuple != "timestamp"
            for rows, value in obs.items()
            for idx, item in enumerate(value)
        }
        for step in ep["sequence"]
    ]

    dict_ep = {k: v for d in dict_observations for k, v in d.items()}

    ser_decoded = pd.Series(dict_ep)
    ser_decoded.index.names = [
        "vehicle",
        "driver",
        "episodestart",
        "timestamp",
        "qtuple",
        "rows",
        "idx",
    ]
    df_decoded = ser_decoded.unstack(level=["qtuple", "rows", "idx"])
    df_decoded.sort_index(inplace=True, axis=1)
    df_episodes_list.append(df_decoded)
    # df_decoded

# try:
#     df_episodes = reduce(
#         lambda left,right: pd.concat([left,right], axis=0, ignore_index=False),df_episodes_list
#     )
# except Exception as e:
#     print(e)

index_names = df_episodes_list[0].index.names
df_episodes = pd.concat(
    df_episodes_list, keys=range(len(df_episodes_list)), names=["batch"] + index_names
)

df_episodes

qtuple                                                                                        action  \
rows                                                                                              r0   
idx                                                                                                0   
batch vehicle driver        episodestart                     timestamp                                 
0     VB7     wang-cheng    2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00   12.0   
                                                             2023-09-28 15:41:59.237642+08:00   12.0   
                                                             2023-09-28 16:41:59.237642+08:00   12.0   
                                                             2023-09-28 17:41:59.237642+08:00   12.0   
                                                             2023-09-28 18:41:59.237642+08:00   12.0   
1     VB7     zheng-longfei 2023-09-26 21:41:59.297357+08:00 2023-09-28 15:41:59.237642+08:00   12.0   
                                                             2023-09-28 17:41:59.237642+08:00   12.0   
2     MP73    wang-cheng    2023-09-23 21:41:59.297357+08:00 2023-09-28 15:41:59.237642+08:00   12.0   
                                                             2023-09-28 16:41:59.237642+08:00   12.0   
                                                             2023-09-28 17:41:59.237642+08:00   12.0   
                                                             2023-09-28 18:41:59.237642+08:00   12.0   
3     VB7     wang-cheng    2023-09-22 21:41:59.297357+08:00 2023-09-28 15:41:59.237642+08:00   12.0   
                                                             2023-09-28 17:41:59.237642+08:00   12.0   
4     MP73    zheng-longfei 2023-09-21 21:41:59.297357+08:00 2023-09-28 15:41:59.237642+08:00   12.0   
                                                             2023-09-28 16:41:59.237642+08:00   12.0   
                                                             2023-09-28 17:41:59.237642+08:00   12.0   
                                                             2023-09-28 18:41:59.237642+08:00   12.0   
5     MP73    zheng-longfei 2023-09-19 22:54:27.750917+08:00 2023-09-27 00:54:27.676989+08:00   12.0   
                                                             2023-09-27 01:54:27.676989+08:00   12.0   
                                                             2023-09-27 02:54:27.676989+08:00   12.0   
                                                             2023-09-27 03:54:27.676989+08:00   12.0   

qtuple                                                                                               \
rows                                                                                                  
idx                                                                                               1   
batch vehicle driver        episodestart                     timestamp                                
0     VB7     wang-cheng    2023-09-28 13:41:59.237642+08:00 2023-09-28 14:41:59.237642+08:00  13.0   
                                                             2023-09-28 15:41:59.237642+08:00  13.0   
                                                             2023-09-28 16:41:59.237642+08:00  13.0   
                                                             2023-09-28 17:41:59.237642+08:00  13.0   
                                                             2023-09-28 18:41:59.237642+08:00  13.0   
1     VB7     zheng-longfei 2023-09-26 21:41:59.297357+08:00 2023-09-28 15:41:59.237642+08:00  13.0   
                                                             2023-09-28 17:41:59.237642+08:00  13.0   
2     MP73    wang-cheng    2023-09-23 21:41:59.297357+08:00 2023-09-28 15:41:59.237642+08:00  13.0   
                                                             2023-09-28 16:41:59.237642+08:00  13.0   
                                                            

In [31]:
len(df_episodes_list)

6

In [32]:
import tensorflow as tf

# from keras.preprocessing.sequence import pad_sequences
# idx = pd.IndexSlice
# episodestart_index = df_episodes.index.unique(level='episodestart')
# episodestart_index

# array of rewards for minibatch
# batch = df_episodes.sort_index(inplace=False, axis=1)
idx = pd.IndexSlice
# df_rewards = batch.loc[
#     :, idx['reward', 'work']
# ]
rewards_list = [
    df_episodes.loc[idx[i, :, :, :, :], idx["reward", "work"]].values.tolist()
    for i in df_episodes.index.levels[0]
]
r_n_t = tf.keras.preprocessing.sequence.pad_sequences(
    rewards_list, padding="post", dtype=np.float32, value=-10000.0
)
print(f"r_n_t: {rewards_list} {r_n_t.shape}")

# array of states for minibatch
states_list = [
    df_episodes.loc[idx[i, :, :, :, :], idx["state", ["velocity", "thrust", "brake"]]].values.tolist()  # type: ignore
    for i in df_episodes.index.levels[0]
]
s_n_t = tf.keras.preprocessing.sequence.pad_sequences(
    states_list, padding="post", dtype=np.float32, value=-10000.0
)
print(f"s_n_t: {states_list} {s_n_t.shape}")

# array of actions for minibatch
actions_list = [
    df_episodes.loc[idx[i, :, :, :, :], idx["action", torque_table_row_names]].values.tolist()  # type: ignore
    for i in df_episodes.index.levels[0]
]
a_n_t = tf.keras.preprocessing.sequence.pad_sequences(
    actions_list, padding="post", dtype=np.float32, value=-10000.0
)
print(f"a_n_t: {actions_list} {a_n_t.shape}")

# array of next_states for minibatch
nstates_list = [
    df_episodes.loc[idx[i, :, :, :, :], idx["nstate", ["velocity", "thrust", "brake"]]].values.tolist()  # type: ignore
    for i in df_episodes.index.levels[0]
]
ns_n_t = tf.keras.preprocessing.sequence.pad_sequences(
    nstates_list, padding="post", dtype=np.float32, value=-10000.0
)

print(f"ns_n_t: {nstates_list} {ns_n_t.shape}")

r_n_t: [[[27.0], [27.0], [27.0], [27.0], [27.0]], [[27.0], [27.0]], [[27.0], [27.0], [27.0], [27.0]], [[27.0], [27.0]], [[27.0], [27.0], [27.0], [27.0]], [[27.0], [27.0], [27.0], [27.0]]] (6, 5, 1)
s_n_t: [[[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]], [[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]], [[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]], [[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0

In [34]:
from eos.data_io.utils.eos_pandas import (
    decode_episode_dataframes_to_padded_arrays_mongo,
)

s, a, r, ns = decode_episode_dataframes_to_padded_arrays_mongo(
    df_episodes, torque_table_row_names, padding_value=-10000
)
s
a
r
ns

array([[[ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01],
        [ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01],
        [ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01],
        [ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01],
        [ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01]],

       [[ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01],
        [ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01],
        [-1.0e+04, -1.0e+

array([[[    12.,     13.,     14.,     15.,     16.,     17.,     18.,
             19.,     20.,     21.,     22.,     23.,     24.,     25.,
             26.],
        [    12.,     13.,     14.,     15.,     16.,     17.,     18.,
             19.,     20.,     21.,     22.,     23.,     24.,     25.,
             26.],
        [    12.,     13.,     14.,     15.,     16.,     17.,     18.,
             19.,     20.,     21.,     22.,     23.,     24.,     25.,
             26.],
        [    12.,     13.,     14.,     15.,     16.,     17.,     18.,
             19.,     20.,     21.,     22.,     23.,     24.,     25.,
             26.],
        [    12.,     13.,     14.,     15.,     16.,     17.,     18.,
             19.,     20.,     21.,     22.,     23.,     24.,     25.,
             26.]],

       [[    12.,     13.,     14.,     15.,     16.,     17.,     18.,
             19.,     20.,     21.,     22.,     23.,     24.,     25.,
             26.],
        [    12.,   

array([[[    27.],
        [    27.],
        [    27.],
        [    27.],
        [    27.]],

       [[    27.],
        [    27.],
        [-10000.],
        [-10000.],
        [-10000.]],

       [[    27.],
        [    27.],
        [    27.],
        [    27.],
        [-10000.]],

       [[    27.],
        [    27.],
        [-10000.],
        [-10000.],
        [-10000.]],

       [[    27.],
        [    27.],
        [    27.],
        [    27.],
        [-10000.]],

       [[    27.],
        [    27.],
        [    27.],
        [    27.],
        [-10000.]]], dtype=float32)

array([[[    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.],
        [    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.],
        [    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.],
        [    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.],
        [    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.]],

       [[    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.],
        [    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.],
        [-10000., -10000., -10000., -10000., -10000., -10000., -10000.,
         -10000., -10000., -10000., -

In [ ]:
r_n_t
a_n_t
s_n_t
ns_n_t

# sample

In [ ]:
from dask.bag import random

print(f"partitions: {b_episodes_read.npartitions}")
# bag = random.choices(b_episodes_read,16, split_every=4)
bag = random.sample(b_episodes_read, 3, split_every=6)
print(f"partitions: {bag.npartitions}")
dicts = bag.compute()
print(f"length: {len(dicts)}")
for b in dicts:
    b["episodestart"] = pd.to_datetime(b["episodestart"], unit="us")
    print(
        f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} "
    )

In [ ]:
from eos.data_io.utils.eos_pandas import avro_ep_decoding

In [ ]:
df_episodes_list = avro_ep_decoding(bag.compute())
try:
    df_episodes = reduce(
        lambda left, right: pd.concat([left, right], axis=0, ignore_index=False),
        df_episodes_list,
    )
except Exception as e:
    print(e)

df_episodes.sort_index(inplace=True)
df_episodes

In [ ]:
dicts = b_episodes_read.compute()
print(f"length: {len(dicts)}")
for b in dicts:
    b["episodestart"] = pd.to_datetime(b["episodestart"], unit="us")
    print(
        f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} "
    )

In [ ]:
from pandas.tseries.offsets import DateOffset

ts0 = ts - DateOffset(days=5, hours=1)
ts1 = ts + DateOffset(hours=1)
type(ts)
print(ts0, ts1)
ts0.timestamp()
ts0

In [ ]:
queried = b_episodes_read.filter(
    lambda x: pd.to_datetime(x["meta"]["episode_meta"]["episodestart"], unit="us") > ts0
)
# bag = random.choices(queried,7)
print(f"ts0:{ts0}")
dicts = queried.compute()
for b in dicts:
    b["episodestart"] = pd.to_datetime(b["episodestart"], unit="us")
    # pd.to_datetime(b['episodestart'],unit='us')
    print(
        f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} "
    )

In [ ]:
# queried = b_episodes_read.filter(lambda x: x['meta']['episode_meta']['vehicle'] == 'VB7' and
#                                         x['meta']['episode_meta']['driver'] == 'wang-cheng' and
#                                         pd.to_datetime(x['meta']['episode_meta']['episodestart'],unit='us') > ts0)

print(ts0, ts1)
queried = b_episodes_read.filter(
    lambda x: x["meta"]["episode_meta"]["driver"] == "wang-cheng"
    and ts0 < pd.to_datetime(x["meta"]["episode_meta"]["episodestart"], unit="us") < ts1
)
# bag = random.choices(queried,7)
dicts = queried.compute()
for b in dicts:
    b["episodestart"] = pd.to_datetime(b["episodestart"], unit="us")
    # pd.to_datetime(b['episodestart'],unit='us')
    print(
        f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} "
    )

In [ ]:
import dask.bag as db

a = db.from_sequence([1, 2, 3])
b = db.from_sequence([4, 5, 6, 1, 2, 3])
c = db.concat([a, b])
c.compute()
c.distinct().compute()

# filter by sequence length of the episode for efficient training

In [ ]:
queried = b_episodes_read.filter(lambda x: len(x["sequence"]) <= 3)
# bag = random.choices(queried,7)
print(f"ts0:{ts0}")
dicts = queried.compute()
for b in dicts:
    b["episodestart"] = pd.to_datetime(b["episodestart"], unit="us")
    # pd.to_datetime(b['episodestart'],unit='us')
    print(
        f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} "
    )